In [ ]:
%reset -f

In [ ]:
import jax
import jax.numpy as jnp
import jax.random as jr

## Data

Des dimensions fixées:

In [ ]:
INP_DIM = 2
OUT_DIM = 3

Une fonction au pif. On va essayer d'ajuster un réseau de neurone à cette fonction.

In [ ]:
def target_fn(inpV):
    outV0=jnp.sin(5*inpV[:,0]) * jnp.cos(8*inpV[:,1])
    outV1=jnp.sin(2*inpV[:,0]) + jnp.cos(4*inpV[:,1])**2
    outV2=jnp.sin(2*inpV[:,0]) - jnp.cos(7*inpV[:,1])*4
    return jnp.stack([outV0,outV1,outV2],axis=1)


target_fn(jnp.zeros([7,INP_DIM])).shape

## Définition du modèle



In [ ]:
def model_fnm(layer_widths):

    def model_init(rkey):
        params = []
        for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
            rk,rkey=jr.split(rkey)
            params.append(
                {"weight":jr.normal(rk,shape=(n_in, n_out))*jnp.sqrt(2/n_in),
                "bias":jnp.zeros([n_out])})
        return params

    def model_apply(params, inp):
        *hidden, last = params
        for layer in hidden:
            inp = jax.nn.relu(inp @ layer['weight'] + layer['bias'])
        return inp @ last['weight'] + last['bias']

    return model_init,model_apply

***A vous:*** ajoutez un appel du modèle pour finir le test.

## Définition de l'optimiseur



In [ ]:
import optax

learning_rate = 0.01
optimizer = optax.adam(learning_rate)

## Définition de la fonction d'entraînement



In [ ]:
@jax.jit
def loss_fn(params,inpV, outV_true):
    outV_pred = model_apply(params, inpV)
    return jnp.mean((outV_pred - outV_true)**2)

In [ ]:
@jax.jit
def train_step(params, opt_state, inpV, outV_true):
    """Performs a single training step."""
    loss_value, grads = jax.value_and_grad(loss_fn)(params,inpV, outV_true)

    updates, new_opt_state = optimizer.update(grads, opt_state)

    new_params = jax.tree.map(lambda x,y:x+y,params,updates)
    #ou bien:
    #new_params = optax.apply_updates(params, updates)

    return new_params, new_opt_state, loss_value

Notes:

* l'optimiseur a lui aussi ses variables propres. L'ensemble de ses variables est appelé 'state'. Dans les autres lib on utiliserais un attribut `optimizer.state` que l'on mettrait à jour de manière `inplace` (et caché).

* En JAX on veut coder des fonctions pures. Cela donne la syntaxe ci-dessus, qui ne cache rien !

* Certains optimiseurs, comme `adamw` utilisent les paramètres pour calculer l'update. Ainsi la syntaxe devient:

        updates, new_opt_state = optimizer.update(grads, opt_state, params)


***A vous:*** Si on n'avait pas utilisé l'optimiseur 'Adam' mais le simple 'SGD' (descente de gradient de base). Que vaudrait `updates` ? Qu'est-ce qu'il y aurait essentiellement dans `opt_state`

## Boucle d'entrainement

In [ ]:
params=model_init(jr.key(0))
opt_state = optimizer.init(params)

In [ ]:
batch_size = 32
rkey=jr.key(0)
losses=[]

for step in range(1000):
    rkey, subkey = jr.split(rkey)
    inpV=jr.uniform(subkey,(batch_size,INP_DIM))
    outV_true=target_fn(inpV)
    params, opt_state, loss = train_step(params, opt_state, inpV, outV_true)
    losses.append(loss)
    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots()
ax.set_xlabel("steps")
ax.set_ylabel("Loss")
ax.set_yscale("log")
ax.plot(losses);

***A vous:*** Quelle(s) cellule(s) faut-il relancer pour poursuivre l'entrainement ?